In [ ]:
from dotenv import load_dotenv
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_openai import ChatOpenAI
from langchain_community.agent_toolkits import create_sql_agent

load_dotenv()

db = SQLDatabase.from_uri("sqlite:///demo.db")

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

agent_executor = create_sql_agent(
    llm, db = db, agent_type = "openai-tools", verbose = True
)

In [ ]:
from langchain.callbacks.base import BaseCallbackHandler

class SQLHandler(BaseCallbackHandler):
    def __init__(self):
        self.sql_result = None

    def on_agent_action(self, action, **kwargs):
        """Run on agent action. if the tool being used is sql_db_query,
         it means we're submitting the sql and we can 
         record it as the final sql"""

        if action.tool == "sql_db_query":
            self.sql_result = action.tool_input

In [ ]:
query = "De combien le pourcentage de terres arables aménagées pour l’irrigation a évolué en 2019?"
handler = SQLHandler()

In [ ]:
result = agent_executor.invoke({"input": query},
                        {"callbacks": [handler]}
            )
print(">>Handler SQL Result: ", handler.sql_result)